# Problem Statement: "The Phone App"

You are designing a smart phone app

1. Users can subscribe for free, identified by their US phone number without extensions. Provide first and last name.
2. Users can add one or more credits cards to their account. Store zipcode, expiration date, and the CVC.
3. The app has paid add-ons called "Track & Field", "Marathon", and "Sprint", each with a fixed price.
4. A user can purchase each add-on, in which case she must provide a credit card for the purchase. A user cannot purchase the same addon twice.

# Design

In [ ]:
import datajoint as dj

In [ ]:
schema = dj.Schema('app')

In [ ]:
@schema
class Account(dj.Manual):
    definition = """
    phone : bigint unsigned 
    ---
    first_name : varchar(30)
    last_name : varchar(30)
    dob : date
    """


In [ ]:
@schema
class CreditCard(dj.Manual):
    definition = """
    card_number :  bigint unsigned 
    ---
    exp_date : date 
    cvc      : smallint unsigned
    zipcode  : int unsigned       
    -> Account
    """

In [ ]:
@schema
class AddOn(dj.Lookup):
    definition = """
    addon_id : int
    ---
    addon_name : varchar(30)
    price : decimal(5, 2) unsigned
    """
    contents = ((1, "Track & Field", 13.99), (2, "Marathon", 26.2), (3, "Sprint", 100.00))

In [ ]:
AddOn()

In [ ]:
@schema
class Purchase(dj.Manual):
    definition = """
    -> Account
    -> AddOn
    ---
    -> CreditCard
    """

In [ ]:
Purchase()

In [ ]:
dj.Diagram(schema)

# Populate 
Now we will populate the database with fake data

In [ ]:
import random
from tqdm import tqdm
from faker import Faker
fake = Faker()

In [ ]:
# insert one account
Account.insert1(dict(
    phone=fake.random_int(1_000_000_0000, 9_999_999_9999),
    first_name=fake.first_name(),
    last_name=fake.last_name(),
    dob=fake.date_of_birth()))

In [ ]:
# insert many accounts
Account.insert(dict(
    phone=fake.random_int(1_000_000_0000, 9_999_999_9999),
    first_name=fake.first_name(),
    last_name=fake.last_name(),
    dob=fake.date_of_birth()) for _ in range(10000))

In [ ]:
Account()

In [ ]:
# get account ids:
keys = Account.fetch("KEY")

In [ ]:
# insert one credit cards
CreditCard.insert1(
    dict(random.choice(keys), 
         zipcode=random.randint(10000,99999), 
         card_number=int(fake.credit_card_number()),
         cvc=random.randint(1, 999), 
         exp_date=fake.future_date()))

In [ ]:
# insert many credit cards
CreditCard.insert(
    dict(random.choice(keys), 
         zipcode=random.randint(10000,99999), 
         card_number=int(fake.credit_card_number()),
         cvc=random.randint(1, 999), 
         exp_date=fake.future_date()) for _ in range(15000))

In [ ]:
# insert random subsamples. Run this several times to populate more
purchases = (Account * CreditCard * AddOn - Purchase).fetch("KEY")
for _ in tqdm(range(100)):
    try:
        Purchase.insert(random.sample(purchases, 30))
    except dj.errors.DuplicateError:
        continue


In [ ]:
Purchase()